# FCR VTR 

In [1]:
import pandas as pd
from pathlib import Path

def calcular_reiteraciones(input_path, output_path=None, window_days=7, sep=";"):
    """
    Agrega campos de reiteración sin eliminar filas:
      - FECHA_SIGUIENTE
      - DIAS_HASTA_REITERACION
      - FLAG_REITERACION_7D  (1..7 días)
      - FLAG_REITERACION_30D (8..29 días)

    Params
    ------
    input_path : str
    output_path : str | None
    window_days : int (solo para compatibilidad; no afecta los flags nuevos)
    sep : str -> delimitador del CSV (default ';')
    """
    # 1) Leer base completa SIN eliminar filas
    df = pd.read_csv(input_path, sep=sep, encoding="utf-8", low_memory=False)

    # 2) Identificador de cliente
    id_priority = ["CustomerIdentity"
    ]
    cust_id = next((c for c in id_priority if c in df.columns), None)
    if cust_id is None:
        raise ValueError("No se encontró un identificador de cliente adecuado.")

    # 3) Columna de fecha
    date_priority = [
        "Fecha"
    ]
    date_col = next((c for c in date_priority if c in df.columns), None)
    if date_col is None:
        raise ValueError("No se encontró una columna de fecha.")

    # 3a) Parseo robusto de fechas (no elimina filas)
    fecha_try1 = pd.to_datetime(df[date_col], errors="coerce", dayfirst=True, infer_datetime_format=True)
    valid_ratio = fecha_try1.notna().mean()
    if valid_ratio < 0.5:
        # si menos del 50% parseó con dayfirst=True, intentamos al revés
        fecha_try2 = pd.to_datetime(df[date_col], errors="coerce", dayfirst=False, infer_datetime_format=True)
        if fecha_try2.notna().mean() > valid_ratio:
            df["_FECHA_DT_"] = fecha_try2
        else:
            df["_FECHA_DT_"] = fecha_try1
    else:
        df["_FECHA_DT_"] = fecha_try1

    # 4) Preparar columnas destino sobre TODAS las filas
    df["FECHA_SIGUIENTE"] = pd.NaT
    df["DIAS_HASTA_REITERACION"] = 0
    df["FLAG_REITERACION_7D"] = 0
    df["FLAG_REITERACION_30D"] = 0

    # 5) Calcular solo en el subconjunto con fecha válida
    m_valid = df["_FECHA_DT_"].notna()
    if m_valid.any():
        sub = df.loc[m_valid, [cust_id, "_FECHA_DT_"]].copy()
        # Orden por cliente y fecha; intentamos un desempate si existe SequentialId
        if "SequentialId" in df.columns:
            sub = df.loc[m_valid, [cust_id, "_FECHA_DT_", "SequentialId"]].copy()
            sub = sub.sort_values([cust_id, "_FECHA_DT_", "SequentialId"])
        else:
            sub = sub.sort_values([cust_id, "_FECHA_DT_"])

        # Índices originales para devolver los resultados a df
        idx_sorted = sub.index
        # Siguiente fecha por cliente
        sub["FECHA_SIGUIENTE_TMP"] = sub.groupby(cust_id)["_FECHA_DT_"].shift(-1)
        # Días hasta reiteración
        dias = (sub["FECHA_SIGUIENTE_TMP"] - sub["_FECHA_DT_"]).dt.days.fillna(0).astype(int).clip(lower=0)

        # Escribir resultados en df manteniendo todas las filas
        df.loc[idx_sorted, "FECHA_SIGUIENTE"] = sub["FECHA_SIGUIENTE_TMP"].values
        df.loc[idx_sorted, "DIAS_HASTA_REITERACION"] = dias.values

        # Flags según regla: 1..7 y 8..29
        df.loc[idx_sorted, "FLAG_REITERACION_7D"] = dias.between(1, 7).astype(int).values
        df.loc[idx_sorted, "FLAG_REITERACION_30D"] = dias.between(8, 29).astype(int).values

    # 6) Reordenar columnas (nuevas primero)
    new_cols = [
        cust_id, date_col, "FECHA_SIGUIENTE", "DIAS_HASTA_REITERACION",
        "FLAG_REITERACION_7D", "FLAG_REITERACION_30D"
    ]
    other_cols = [c for c in df.columns if c not in new_cols and c != "_FECHA_DT_"]
    out = df[new_cols + other_cols]

    # 7) Guardar CSV si corresponde
    if output_path:
        Path(output_path).parent.mkdir(parents=True, exist_ok=True)
        out.to_csv(output_path, index=False, sep=";")
        print(f"[OK] Base enriquecida guardada en: {output_path}")
        print(f"Filas originales: {len(df)} | Filas en salida: {len(out)} (debe coincidir)")

    # Limpieza de columna auxiliar si te lo quedas en memoria
    out = out.drop(columns=[c for c in ["_FECHA_DT_"] if c in out.columns], errors="ignore")
    return out


# ======================
# EJEMPLO DE USO
# ======================
if __name__ == "__main__":
    INPUT = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Blip Power Bi Base\f_blip_ha.csv"
    OUTPUT_CSV = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Blip Power Bi Base\f_blip_ha_reiteraciones.csv"

    df_resultado = calcular_reiteraciones(
        input_path=INPUT,
        output_path=OUTPUT_CSV,
        window_days=7,
        sep=","  # ¡ojo con el delimitador real de tu archivo!
    )

C:\Users\wduran\AppData\Local\Temp\ipykernel_10956\3403666846.py:38: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  fecha_try1 = pd.to_datetime(df[date_col], errors="coerce", dayfirst=True, infer_datetime_format=True)
C:\Users\wduran\AppData\Local\Temp\ipykernel_10956\3403666846.py:42: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  fecha_try2 = pd.to_datetime(df[date_col], errors="coerce", dayfirst=False, infer_datetime_format=True)


[OK] Base enriquecida guardada en: C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Blip Power Bi Base\f_blip_ha_reiteraciones.csv
Filas originales: 1697301 | Filas en salida: 1697301 (debe coincidir)


In [2]:
# Definir la ruta del archivo correctamente
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Blip Power Bi Base\f_blip_ha_reiteraciones.csv"
base_fcr = pd.read_csv(archivo, delimiter=';', encoding='latin1')  # Prueba con 'latin1'

# Ajustar las opciones para mostrar todas las columnas
pd.set_option('display.max_columns', None)

C:\Users\wduran\AppData\Local\Temp\ipykernel_10956\3903480130.py:3: DtypeWarning: Columns (22,25,26,27,28,29,30,31,32,33,34,35,36,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,105) have mixed types. Specify dtype option on import or set low_memory=False.
  base_fcr = pd.read_csv(archivo, delimiter=';', encoding='latin1')  # Prueba con 'latin1'


In [3]:
base_fcr

,CustomerIdentity,Fecha,FECHA_SIGUIENTE,DIAS_HASTA_REITERACION,FLAG_REITERACION_7D,FLAG_REITERACION_30D,SequentialId,AgentIdentity,Status,StorageDate,ExpirationDate,CloseDate,Team,Closed,Tags,QueueTime,FirstResponseTime,AverageResponseTime,CustomerName,CustomerEmail,CustomerGender,CustomerCity,CustomerPhoneNumber,AgentName,AgentEmail,1,2,3,campaignOriginator,botOrigin,campaignMessageTemplate,campaignId,Menu Hogar,priorityTag,Campaign,Menu Principal,team,rut,activeMessageFileName,actividad,activa,headline,body,source_type,source_id,source_url,media_id,tiporedproductotoa,08-oct,13:00-16:00horas,NumeroSerie,30-ago,16:00-19:00horas,SourceId,SourceUrl,SourceTitle,SourceType,25-sept,direccion,nombreplanprincipal,svatotal,databeneficiosincluidos,preciopermanente,dctomensual,montomensualplan,totalproductoadicional,07-oct,10:00-13:00horas,21-oct,oct-13,27-sept,variable1,23-oct,13:00-16:00,09-oct,20-nov,nombre,mes,montopago,numboleta,fechapago,fechacorte,11-oct,1db07bec29ded479f93733c484b4cb7ef,cargomensual,montomensual,23-ago,16:00-19:00,4,5,6,7,28-ago,urlarr[1],10-jul,14-feb,HORA,INTERVALO,NEUTRA,Semana,Def_IBR,Llave,Flag_Outbound,MAIL,Flag_Dotacion_Konecta,Rama,Defle_meta
0,9da90e55-3250-40e6-b319-63ad4d2fe2ba@tunnel.ms...,2025-03-06,NaN,0,0,0,2357422,KEVINANDRES.MORENOMAFLA%40E.VTR.CL@blip.ai,Transferred,2025-03-06 23:58:28-03:00,NaN,2025-03-06 23:59:02 -03:00,tecnico_fijo,True,NaN,00:00:00,00:00:16,NaN,Cuenta Empresa,NaN,NaN,NaN,56958024590,Kevin Moreno,kevinandres.morenomafla@e.vtr.cl,11749801-8,https://claro.globalcommerce.uy/img/RedNeutra2...,Document,oct-13,RIO BLANCO 1048. La Granja,NaN,NaN,ALTA,jfe/form/SV_2nsxPhTGlFpMkwS?Q_CHL=email&Q_DL=E...,e9f50315-88a1-4b8f-a602-8ed8562e2623,encuesta_satisfaccion_fase2_v3,javier.cobos@clarovtr.cl,11749801-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:58:28,23:00,0,1,0,56958024590_2025-03-06,1,KEVINANDRES.MORENOMAFLA,0,NaN,0
1,b82712ac-a512-40df-aacb-0d50586d7f55@tunnel.ms...,2025-03-06,2025-03-07,1,1,0,2357418,zharick.quinterobarreto%40ext.clarovtr.cl@blip.ai,Transferred,2025-03-06 23:55:19-03:00,NaN,2025-03-07 00:00:02 -03:00,tecnico_fijo,True,NaN,00:00:00,00:00:15,NaN,Paty,NaN,NaN,NaN,56971249929,Zharick Quintero,zharick.quinterobarreto@ext.clarovtr.cl,NaN,https://imgur.com/vtr-fibra-FzoyQPn,Document,13:00-16:00,PJE GUILLERMO VALENCIA 4134. Puente Alto,reset,NaN,ALTA,jfe/form/SV_2nsxPhTGlFpMkwS?Q_CHL=email&Q_DL=E...,reset,reset,javier.cobos@clarovtr.cl,Servicio Hogar,Servicio Hogar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:55:19,23:00,0,1,0,56971249929_2025-03-06,0,zharick.quinterobarreto,0,NaN,0
2,096c9eec-3af9-4668-9967-b3ac56eacb8a@tunnel.ms...,2025-03-06,2025-03-07,1,1,0,2357417,michael.piedrahitaduque%40ext.clarovtr.cl@blip.ai,Transferred,2025-03-06 23:55:07-03:00,NaN,2025-03-06 23:59:56 -03:00,comercial_fijo,True,NaN,00:00:00,00:00:21,163.0,??,NaN,NaN,NaN,56999289783,michael.piedrahitaduque@ext.clarovtr.cl piedra...,michael.piedrahitaduque@ext.clarovtr.cl,16847027-4,https://claro.globalcommerce.uy/img/RedNeutra2...,Document,oct-13,CACHAPOAL 3725. Temuco,NaN,NaN,NaN,jfe/form/SV_2nsxPhTGlFpMkwS?Q_CHL=email&Q_DL=E...,33967cb2-936f-4c95-a42f-dedfcf3250a1,encuesta_satisfaccion_fase2_v3,javier.cobos@clarovtr.cl,16847027-4,Servicio Hogar,Mi Plan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:55:07,23:00,0,1,0,56999289783_2025-03-06,1,michael.piedrahitaduque,0,NaN,0
3,3340eb7e-009d-4926-bfa6-352f29f62df0@tunnel.ms...,2025-03-06,2025-03-07

In [85]:
# Aseguramos que la columna Fecha sea datetime (ya viene en formato YYYY-MM-DD)
base_fcr["Fecha"] = pd.to_datetime(base_fcr["Fecha"], errors="coerce")

# Filtrar solo registros de enero 2025
base_fcr_m = base_fcr[
    (base_fcr["Fecha"].dt.year == 2025) & (base_fcr["Fecha"].dt.month == 8)
]

In [86]:
base_fcr_m

,CustomerIdentity,Fecha,FECHA_SIGUIENTE,DIAS_HASTA_REITERACION,FLAG_REITERACION_7D,FLAG_REITERACION_30D,SequentialId,AgentIdentity,Status,StorageDate,ExpirationDate,CloseDate,Team,Closed,Tags,QueueTime,FirstResponseTime,AverageResponseTime,CustomerName,CustomerEmail,CustomerGender,CustomerCity,CustomerPhoneNumber,AgentName,AgentEmail,1,2,3,campaignOriginator,botOrigin,campaignMessageTemplate,campaignId,Menu Hogar,priorityTag,Campaign,Menu Principal,team,rut,activeMessageFileName,actividad,activa,headline,body,source_type,source_id,source_url,media_id,tiporedproductotoa,08-oct,13:00-16:00horas,NumeroSerie,30-ago,16:00-19:00horas,SourceId,SourceUrl,SourceTitle,SourceType,25-sept,direccion,nombreplanprincipal,svatotal,databeneficiosincluidos,preciopermanente,dctomensual,montomensualplan,totalproductoadicional,07-oct,10:00-13:00horas,21-oct,oct-13,27-sept,variable1,23-oct,13:00-16:00,09-oct,20-nov,nombre,mes,montopago,numboleta,fechapago,fechacorte,11-oct,1db07bec29ded479f93733c484b4cb7ef,cargomensual,montomensual,23-ago,16:00-19:00,4,5,6,7,28-ago,urlarr[1],10-jul,14-feb,HORA,INTERVALO,NEUTRA,Semana,Def_IBR,Llave,Flag_Outbound,MAIL,Flag_Dotacion_Konecta,Rama,Defle_meta
1601030,466811df-d15a-4aba-8b22-578a56a69f7b@tunnel.ms...,2025-08-21,NaN,0,0,0,96615,estefano.valladaressalinas%40ext.clarochile.cl...,Transferred,2025-08-21 23:53:02-04:00,NaN,2025-08-22 00:00:34 -04:00,tecnico hogar,True,NaN,00:00:01,00:00:08,168.0,Yani CortÃ©s,NaN,NaN,NaN,56931059401,estefano.valladaressalinas@ext.clarochile.cl,estefano.valladaressalinas@ext.clarochile.cl,url-tracker/OFF-hJq_7E,NaN,18661491-7,True,reset,NaN,NaN,NaN,reset,NaN,NaN,NaN,18661491-7,NaN,reset,Hola,tecnico_hogar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:53:02,23:00,0,3,0,56931059401_2025-08-21,0,estefano.valladaressalinas,0,NaN,0
1601031,6f59bc0d-72b0-4ddf-8170-181ab17e7aba@tunnel.ms...,2025-08-21,NaN,0,0,0,96614,daniela.mirandarios%40ext.clarochile.cl@blip.ai,Transferred,2025-08-21 23:51:04-04:00,NaN,2025-08-22 00:00:22 -04:00,comercial hogar,True,NaN,00:00:00,00:00:04,174.0,Pablo Huichapan,NaN,NaN,NaN,56939241715,daniela.mirandarios@ext.clarochile.cl,daniela.mirandarios@ext.clarochile.cl,NaN,NaN,13929432-7,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13929432-7,NaN,No Campaign,"Hola, necesito ayuda con mis servicios de Clar...",comercial_hogar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:51:04,23:00,0,3,0,56939241715_2025-08-21,0,daniela.mirandarios,0,NaN,0
1601032,5c09e048-dd6a-4997-9fd2-5c6efd711b0d@tunnel.ms...,2025-08-21,NaN,0,0,0,96613,alexis.mondragonrequejo%40e.vtr.cl@blip.ai,ClosedAttendant,2025-08-21 23:49:59-04:00,NaN,2025-08-21 23:55:38 -04:00,default,True,"[""FACTURACION""]",00:00:00,00:00:10,NaN,??,NaN,NaN,NaN,56956083531,alexis.mondragonrequejo@e.vtr.cl Mondragon,alexis.mondragonrequejo@e.vtr.cl,NaN,NaN,16801653-0,True,NaN,Inactividad,NaN,NaN,NaN,NaN,NaN,NaN,16801653-0,NaN,No Campaign,"Hola, necesito ayuda con mis servicios de Clar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:49:59,23:00,0,3,0,56956083531_2025-08-21,0,alexis.mondragonrequejo,0,NaN,0
1601033,b802d67e-0a6d-4353-b3e8-bb95664c8d0c@tunnel.ms...,2025-08-21,NaN,0,0,0,96612,josue.borjasnatividad%40ext.clarochile.cl@blip.ai,ClosedClientInactivity,2025-08-21 23:42:40-04:00,NaN,2025-08-21 23:53:32 -04:00,tecnico hogar,True,NaN,00:00:00,00:00:14,86.0,Mercy Molina,NaN,NaN,NaN,56954249930,Josue Borjas,josue.borjasnatividad@ext.clarochile.cl,NaN,NaN,21201479-6,True,NaN,Inactividad,NaN,NaN,reset,NaN,NaN,N

In [87]:
# Define the file path
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Dotaciones\VTR\Dotación_VTR_20250815.xlsx"
# Read the specific sheet named 'Dotacion' from the Excel file
dotacion = pd.read_excel(archivo, sheet_name='Dotacion')  # Specify the sheet name

In [88]:
dotacion_filtrada = dotacion[
    (dotacion['ESTADO_CONTRATO'].str.lower() == 'vigente') &
    (dotacion['PLATAFORMA'].str.lower().isin(['0 wsp. oopp', '0 wsp postventa'])) &
    (dotacion['CARGO'].str.lower() == 'asesor') &
    (dotacion['ESTADO_ASESOR'].str.lower().isin(['activo', 'licencia', 'vacaciones']))
]

In [89]:
# Crear la nueva columna MAIL que extrae la parte izquierda del arroba
dotacion_filtrada['MAIL'] = dotacion_filtrada['CORREO_ELECTRONICO'].str.split('@').str[0]

C:\Users\wduran\AppData\Local\Temp\ipykernel_10956\3410713443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dotacion_filtrada['MAIL'] = dotacion_filtrada['CORREO_ELECTRONICO'].str.split('@').str[0]


In [90]:
# Creamos versiones en minúscula del campo MAIL para el join
dotacion_filtrada['MAIL'] = dotacion_filtrada['MAIL'].str.lower()
base_fcr_m['MAIL'] = base_fcr_m['MAIL'].str.lower()

C:\Users\wduran\AppData\Local\Temp\ipykernel_10956\3461589852.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dotacion_filtrada['MAIL'] = dotacion_filtrada['MAIL'].str.lower()
C:\Users\wduran\AppData\Local\Temp\ipykernel_10956\3461589852.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_fcr_m['MAIL'] = base_fcr_m['MAIL'].str.lower()


In [91]:
# Hacer merge directo por MAIL
base_fcr_m = base_fcr_m.merge(
    dotacion_filtrada[['MAIL', 'EMPRESA_PAIS']], 
    on='MAIL', 
    how='left'
)

# Crear campo MARCA: 'VTR' si coincide, vacío si no
base_fcr_m['MARCA'] = base_fcr_m['EMPRESA_PAIS'].notna().map({True: 'VTR', False: ''})

In [92]:
# Eliminar filas con NaN en EMPRESA_PAIS
base_fcr_m = base_fcr_m[base_fcr_m['EMPRESA_PAIS'].notna()].copy()

In [93]:
base_fcr_m

,CustomerIdentity,Fecha,FECHA_SIGUIENTE,DIAS_HASTA_REITERACION,FLAG_REITERACION_7D,FLAG_REITERACION_30D,SequentialId,AgentIdentity,Status,StorageDate,ExpirationDate,CloseDate,Team,Closed,Tags,QueueTime,FirstResponseTime,AverageResponseTime,CustomerName,CustomerEmail,CustomerGender,CustomerCity,CustomerPhoneNumber,AgentName,AgentEmail,1,2,3,campaignOriginator,botOrigin,campaignMessageTemplate,campaignId,Menu Hogar,priorityTag,Campaign,Menu Principal,team,rut,activeMessageFileName,actividad,activa,headline,body,source_type,source_id,source_url,media_id,tiporedproductotoa,08-oct,13:00-16:00horas,NumeroSerie,30-ago,16:00-19:00horas,SourceId,SourceUrl,SourceTitle,SourceType,25-sept,direccion,nombreplanprincipal,svatotal,databeneficiosincluidos,preciopermanente,dctomensual,montomensualplan,totalproductoadicional,07-oct,10:00-13:00horas,21-oct,oct-13,27-sept,variable1,23-oct,13:00-16:00,09-oct,20-nov,nombre,mes,montopago,numboleta,fechapago,fechacorte,11-oct,1db07bec29ded479f93733c484b4cb7ef,cargomensual,montomensual,23-ago,16:00-19:00,4,5,6,7,28-ago,urlarr[1],10-jul,14-feb,HORA,INTERVALO,NEUTRA,Semana,Def_IBR,Llave,Flag_Outbound,MAIL,Flag_Dotacion_Konecta,Rama,Defle_meta,EMPRESA_PAIS,MARCA
14555,58fdc2a2-6388-4152-9c78-b58f01d13bfc@tunnel.ms...,2025-08-21,NaN,0,0,0,3099728,ADRIAN.ARIASZENA%40E.VTR.CL@blip.ai,ClosedClientInactivity,2025-08-21 23:57:31-04:00,NaN,2025-08-22 00:05:34 -04:00,tecnico_fijo,True,"[""inactividad""]",00:00:00,00:00:28,65.0,Werner Stehr,NaN,NaN,NaN,56994454512,Adrian Arias,adrian.ariaszena@e.vtr.cl,NaN,NaN,NaN,a65336b8-3e11-45dc-aeee-dd30f8ed85b8,cambio_red_v2,franco.vargas@clarovtr.cl,4847643-0,Servicio Hogar,Servicio Hogar,NaN,NaN,NaN,4847643-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:57:31,23:00,0,3,0,56994454512_2025-08-21,0,adrian.ariaszena,1,NaN,0,KONECTA PERU,VTR
14557,a5c8fba2-4990-4f5d-aa0e-34d48a70c19c@tunnel.ms...,2025-08-21,NaN,0,0,0,3099726,jhonatan.aristizabal%40e.vtr.cl@blip.ai,Transferred,2025-08-21 23:52:40-04:00,NaN,2025-08-22 00:00:05 -04:00,comercial_fijo,True,NaN,00:00:00,00:00:58,NaN,Alejandro,NaN,NaN,NaN,56954629319,David Aristizabal,jhonatan.aristizabal@e.vtr.cl,NaN,reset,NaN,NaN,NaN,NaN,20713944-0,Servicio Hogar,Servicio Hogar,NaN,NaN,NaN,20713944-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:52:40,23:00,0,3,0,56954629319_2025-08-21,0,jhonatan.aristizabal,0,NaN,0,ATENTO COLOMBIA,VTR
14558,859aeb8e-b1c1-4f9d-a013-64d7ae5895d9@tunnel.ms...,2025-08-21,NaN,0,0,0,3099725,YORANIA.PESCADORLARGO%40ext.clarovtr.cl@blip.ai,ClosedAttendant,2025-08-21 23:49:42-04:00,NaN,2025-08-21 23:54:35 -04:00,tecnico_fijo,True,"[""Visita TÃ©cnica reag/desi/incum""]",00:00:00,00:00:22,NaN,Lorena,NaN,NaN,NaN,56946991584,Yorania Pescador Largo,yorania.pescadorlargo@ext.clarovtr.cl,Inactividad,reset,NaN,NaN,NaN,NaN,5356584-0,Servicio Hogar,Servicio Hogar,NaN,NaN,NaN,5356584-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:49:42,23:00,0,3,0,56946991584_2025-08-21,0,yorania.pescadorlargo,0,NaN,0,ATENTO COLOMBIA,VTR
14559,b048b60f-6a40-497c-a763-15eed4fff545@tunnel.ms...,2025-08-21,NaN,0,0,0,3099724,MARIA.CHICACATANO%40e.vtr.cl@blip.ai,Transferred,2025-08-21 23:45:32-04:00,NaN,2025-08-22 00:00:30 -04:00,comercial_fijo,True,NaN,00:00:00,00:00:15,197.0,Nicol Ibar,NaN,NaN,NaN,56978647046,MARIA CHICA,maria.chicacatano@e.vtr.cl,NaN,reset,NaN,reset,reset,zoila.sanchez@e.vtr.cl,20209934-3,Servicio Hogar,Consulta Saldo,oct-13,AGUSTIN AVEZON 1075 EDIF. B DEPTO 120

In [94]:
# Ahora sí, guardás el archivo
base_fcr_m.to_csv(r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\FCR\Base_Consolidada_FCR_AGO.csv", index=False)